## Podsumowanie tekstu

In [1]:
from docx import Document
import docx
import re
import pandas as pd
#import nltk
#import heapq
import operator
import csv

st_list = list()
with open('sw_polski.csv', newline='', encoding='windows-1250') as csvfile:
    plik = csv.reader(csvfile)
    for row in plik:
        st_list.append(row[0])

In [16]:
class summarizer():
    
    def _init_(self):
        self.text_to_score=''
        self.text_to_model=''
                
    def summarize_please(self, file_path: str, impsents: int = 10, dropwordsfromdict: int = 0, sentlen2score: int = 27 ):
        
        try:
            doc = Document(file_path)
        except:
            print('Brak pliku lub niewłaściwe rozszerzenie.\nUpewnij się, że śieżka jest poprawna a plik ma rozszezrenie "docx".')
            return None
        
        print('Przetwarzanie dokumentu...')
        
        self.text_to_score = self.txt_prep_sc(doc)
        self.text_to_model = self.txt_prep_md(self.text_to_score)
        
        print('Ocena tekstu... \n')
        
        self.sents_dict = self.scoring(dropwordsfromdict, sentlen2score)
                
        self.text_summary(impsents)
        
        
     

    def scoring(self, dropwordsfromdict, sentlen2score ):
        word2count = {}
        
        for word in self.text_to_model.split():
            if word not in st_list:
                if word not in word2count.keys():
                    word2count[word] = 1
                else:
                    word2count[word] += 1
                    
        if dropwordsfromdict>0:
            for el in sorted(word2count.items(), key=operator.itemgetter(1), reverse=1 )[0:dropwordsfromdict-1]:
                del word2count[el[0]]
        
        for key in word2count.keys():
            word2count[key] = word2count[key]/max(word2count.values())
            
        sent_dict = {}
        sents = [sent.strip() for sent in self.text_to_score.split('.') if (not sent=='' and not sent==' ')]
        
        for sentence in sents:
            for word in sentence.split():
                if word.lower() in word2count.keys():
                    if len(sentence.split()) < sentlen2score:
                        if sentence not in sent_dict.keys():
                            sent_dict[sentence] = word2count[word.lower()]
                        else:
                            sent_dict[sentence] += word2count[word.lower()]
        return sent_dict
   

    def text_summary(self, sent_count: int):
        print('Najistotniejsze zdania to:', '\n')
        for el in sorted(self.sents_dict.items(), key=operator.itemgetter(1), reverse=1 )[0:sent_count]:
            print('{}.\n'.format(el[0]) )
    
    
    @staticmethod
    def txt_prep_sc(doc: docx.document.Document):
        doc_text=''
        dot_clean = ('np.', 'tzw.', ' ust.', 'tj.' )
        space_rep = ('\\xa0', '[„”]', '\s+' )
            
        for par in doc.paragraphs:
            if (len(par.text)>2):
                if (par.text.strip()[-1] == ',' or par.text.strip()[-1]==':' or par.text.strip()[-3:]=='lub'):
                    doc_text+= par.text + ' '
                else:
                    doc_text+= par.text + '. '
        
        doc_text = re.sub('\\n', '. ', doc_text)
        doc_text = re.sub('\.\.','.', doc_text)

        for el in dot_clean:
            doc_text = re.sub(el, el[0:-1], doc_text)
            
        for el in space_rep:
            doc_text = re.sub(el, ' ', doc_text)
            
        return doc_text
        
        
    @staticmethod
    def txt_prep_md(text: str):
        text_mod = re.sub('[()§.,;:\-%\"—–„”]', ' ', text).lower()
        text_mod = re.sub('\d', ' ', text_mod)
        text_mod = re.sub(' \w ', ' ', text_mod)
        text_mod = re.sub('\s+', ' ', text_mod)
        
        return text_mod

In [17]:
a = summarizer()

### Przykładowe wywołania

#### summarize_please(file_path: str, impsents: int = 10, dropwordsfromdict: int = 0, sentlen2score: int = 27 )

#### file_path 
obvious

#### impsents
liczba zdań, które chcemy wyświetlić

#### dropwordsfromdict 
liczba najczęściej występujących słów, które chcemy usunąć ze słownika (usunięcie tych słów może wpłynąć na jakość podsumowania)

#### sentlen2score
maksymalna długość zdania, które chcemy oceniać (zbyt długie zdania mogą być fałśzywie oceniane jako informacyjne)

In [1]:
a.summarize_please('doc.docx', sentlen2score=20)

NameError: name 'a' is not defined

In [2]:
a.summarize_please('doc.docx', dropwordsfromdict=0, impsents=15)

NameError: name 'a' is not defined